In [3]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("HUGGING_FACE_API_KEY")

In [ ]:
import requests
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Inference API details
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-base"
headers = {"Authorization": f"Bearer {api_key}"}


def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()


def api_based_llm(input_text):
    response = query({"inputs": input_text})
    return response[0]["generated_text"] if isinstance(response, list) else response


# Example 1: Basic Question-Answering using API
countries = ["France", "Japan", "Australia"]
for country in countries:
    question = f"What is the capital of {country}?"
    response = api_based_llm(question)
    print(f"Q: {question}")
    print(f"A: {response}\n")

# Example 2: Using Prompt Templates
prompt = PromptTemplate(
    input_variables=["country"],
    template="What is the capital of {country}?"
)


Q: What is the capital of France?
A: london

Q: What is the capital of Japan?
A: nagasaki

Q: What is the capital of Australia?
A: melbourne



In [5]:


def chain_invoke_api(prompt, input_var):
    formatted_prompt = prompt.format(**input_var)
    return api_based_llm(formatted_prompt)


countries = ["Sweden", "Mexico", "Vietnam"]
for country in countries:
    response = chain_invoke_api(prompt, {"country": country})
    print(f"Q: {prompt.format(country=country)}")
    print(f"A: {response}\n")

# Example 3: Sequential Chains using API
pt1 = PromptTemplate(
    input_variables=["topic"],
    template="Generate a random question about {topic}: Question: "
)
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}"
)


Q: What is the capital of Sweden?
A: sweden

Q: What is the capital of Mexico?
A: san juan

Q: What is the capital of Vietnam?
A: vietnam



In [6]:

def sequential_chain_api(pt1, pt2, topic):
    generated_question = api_based_llm(pt1.format(topic=topic))
    answer = api_based_llm(pt2.format(question=generated_question))
    print(f"Generated Question: {generated_question}")
    print(f"Answer: {answer}")


# Example topic
sequential_chain_api(pt1, pt2, "an animal")

# Example 4: RAG with PDF (Preserved for context, requires adaptation to API calls)
pdf = './pdf/what is generative ai.pdf'
loaders = [PyPDFLoader(pdf)]

index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
).from_loaders(loaders)

Generated Question: What is the most common way to kill a fox?
Answer: slicing it with a sharp knife


C:\Users\Albert\AppData\Local\Temp\ipykernel_20592\4246269487.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(),
C:\Users\Albert\AppData\Local\Temp\ipykernel_20592\4246269487.py:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),
c:\Users\Albert\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook

c:\Users\Albert\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\indexes\vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [1]:
# # chain = RetrievalQA.from_chain_type(
# #     llm=api_based_llm,
# #     chain_type="stuff",
# #     retriever=index.vectorstore.as_retriever(),
# #     input_key="question"
# # )

# # Test RAG (This will depend on adapting the Retriever to work seamlessly with API calls)
# question = "What is Machine Learning?"
# response = chain.invoke({"question": question})
# print(f"Q: {question}")
# print(f"A: {response['result']}")

In [10]:
import requests
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Define the API endpoint and headers for Hugging Face Inference API
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-base"
headers = {"Authorization": f"Bearer {api_key}"}

# Function to query the language model API


def query_api(prompt):
    response = requests.post(API_URL, headers=headers, json={"inputs": prompt})
    response_data = response.json()
    return response_data[0].get("generated_text", "Error in response") if isinstance(response_data, list) else str(response_data)


# Step 1: Load and index documents (RAG Retrieval)
pdf = './pdf/what-is-generative-ai.pdf'  # Replace with your PDF file
loaders = [PyPDFLoader(pdf)]
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
).from_loaders(loaders)

# Step 2: Retrieve relevant context for a given query


def retrieve_context(query):
    retriever = index.vectorstore.as_retriever()
    retrieved_docs = retriever.get_relevant_documents(query)
    context = " ".join([doc.page_content for doc in retrieved_docs])
    return context

# Step 3: Perform RAG using serverless API-based language model


def perform_rag(query):
    # Retrieve context for the query
    context = retrieve_context(query)
    # Construct the prompt for the API call
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    # Query the language model API for the answer
    response = query_api(prompt)
    return response


# Example query
question = "What is standfor GPT?"
response = perform_rag(question)
print(f"Q: {question}")
print(f"A: {response}")

C:\Users\Albert\AppData\Local\Temp\ipykernel_20592\3867811952.py:24: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),
c:\Users\Albert\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\indexes\vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


Q: What is standfor GPT?
A: generative pretrained transformer


In [11]:
# Example query
question = "What is standfor AI?"
response = perform_rag(question)
print(f"Q: {question}")
print(f"A: {response}")

Q: What is standfor AI?
A: Generative artificial intelligence (AI) describes algorithms (such as ChatGPT) that can


In [1]:


# # Initialize the model and tokenizer
# model_id = "google/flan-ul2"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# # Create the pipeline
# pipe = pipeline(
#     "text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_length=512,
#     temperature=0.2,
#     top_p=1,
#     top_k=25,
#     repetition_penalty=1.0
# )

# # Create LangChain HF pipeline
# local_llm = HuggingFacePipeline(pipeline=pipe)

# # Example 1: Basic Question-Answering
# countries = ["France", "Japan", "Australia"]
# for country in countries:
#     question = f"What is the capital of {country}?"
#     response = local_llm(question)
#     print(f"Q: {question}")
#     print(f"A: {response}\n")

# # Example 2: Using Prompt Templates
# prompt = PromptTemplate(
#     input_variables=["country"],
#     template="What is the capital of {country}?"
# )

# chain = LLMChain(llm=local_llm, prompt=prompt)

# countries = ["Sweden", "Mexico", "Vietnam"]
# for country in countries:
#     response = chain.invoke({"country": country})
#     print(f"Q: {prompt.format(country=country)}")
#     print(f"A: {response['text']}\n")

# # Example 3: Sequential Chains
# pt1 = PromptTemplate(
#     input_variables=["topic"],
#     template="Generate a random question about {topic}: Question: "
# )
# pt2 = PromptTemplate(
#     input_variables=["question"],
#     template="Answer the following question: {question}"
# )

# prompt_to_model_1 = LLMChain(llm=local_llm, prompt=pt1)
# prompt_to_model_2 = LLMChain(llm=local_llm, prompt=pt2)
# qa = SimpleSequentialChain(
#     chains=[prompt_to_model_1, prompt_to_model_2],
#     verbose=True
# )

# # Example topic
# qa.invoke("an animal")

# # Example 4: RAG with PDF
# pdf = 'what-is-generative-ai.pdf'
# loaders = [PyPDFLoader(pdf)]

# index = VectorstoreIndexCreator(
#     embedding=HuggingFaceEmbeddings(),
#     text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# ).from_loaders(loaders)

# chain = RetrievalQA.from_chain_type(
#     llm=local_llm,
#     chain_type="stuff",
#     retriever=index.vectorstore.as_retriever(),
#     input_key="question"
# )

# # Test RAG
# question = "what is Machine Learning?"
# response = chain.invoke({"question": question})
# print(f"Q: {question}")
# print(f"A: {response['result']}")